In [1]:
import keras as K
import keras
from keras import layers
from keras import ops
import tensorflow as tf
import os
import numpy as np
from glob import glob
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm  
# For data preprocessing
from keras import losses

from PIL import Image
from tensorflow import image as tf_image
from tensorflow import data as tf_data
from tensorflow import io as tf_io
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# (2) keras backend 문법 사용 가능

def dice_score_loss(y_true, y_pred):
  numerator = 2. * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)
  return tf.reduce_mean(1 - numerator / denominator)





2024-02-23 14:59:51.473466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 14:59:52.034720: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 8
NUM_CLASSES = 4
train_df = pd.read_csv(
    '../../data/segmentationDDH/train_aug_segmentation_dataset.csv')
val_df = pd.read_csv(
    '../../data/segmentationDDH/val_aug_segmentation_dataset.csv')

train_img_list = train_df['file name'].to_list()
train_label_list = train_df['standard mask'].to_list()
train_case_list = train_df['case'].to_list()
train_img_path = '../../data/segmentationDDH/aug_dataset/train/'
val_img_list = val_df['file name'].to_list()
val_label_list = val_df['standard mask'].to_list()
val_case_list = val_df['case'].to_list()
val_img_path = '../../data/segmentationDDH/aug_dataset/val/'

val_image=np.zeros((len(val_img_list),IMAGE_SIZE,IMAGE_SIZE,3))
val_mask=np.zeros((len(val_img_list),IMAGE_SIZE,IMAGE_SIZE,4),dtype=np.uint8)    
train_image=np.zeros((len(train_img_list),IMAGE_SIZE,IMAGE_SIZE,3))
train_mask=np.zeros((len(train_img_list),IMAGE_SIZE,IMAGE_SIZE,4),dtype=np.uint8)

for i in tqdm(range(len(train_img_list))):
    train_image[i] = np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/image/'+train_img_list[i]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,0]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/1'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,1]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/2'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,2]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/3'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,3]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/4'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))

    

for i in tqdm(range(len(val_img_list))):
    val_image[i] = np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/image/'+val_img_list[i]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,0]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/1.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,1]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/2.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,2]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/3.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,3]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/4.png').resize((IMAGE_SIZE, IMAGE_SIZE)))


  0%|          | 0/20088 [00:00<?, ?it/s]

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return ops.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
        interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)

def dice_coef(y_true, y_pred):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
def dice_coef(y_true, y_pred):
    y_true = K.eval(y_true)
    y_pred = K.eval(y_pred)
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    return (2. * intersection) / (np.sum(np.square(y_true), -1) + np.sum(np.square(y_pred), -1))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [ ]:

with tf.device('/gpu:0'):
    model=DeeplabV3Plus(IMAGE_SIZE,NUM_CLASSES)
    checkpoint_filepath = "../../model/segmentation/DeepLabV3_tf_checkpoints.keras"
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only= True
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss=[dice_coef_loss],metrics=[dice_coef]
    )

    history = model.fit(
        train_image, train_mask, batch_size=BATCH_SIZE, epochs=300, validation_data=(val_image, val_mask),callbacks=model_checkpoint_callback
    )

